In [ ]:
!pip3 install pycrypto

     |████████████████████████████████| 446 kB 5.2 MB/s 
  Created wheel for pycrypto: filename=pycrypto-2.6.1-cp37-cp37m-linux_x86_64.whl size=499932 sha256=5a2c780f3ecc27585d4385ea263bc9dc853b8427cbc03051af50e839ad245c13
  Stored in directory: /root/.cache/pip/wheels/cf/85/ba/bbd7c96add459de7598fb424e5ff2309baf2095c844ac0f191
Successfully built pycrypto


In [ ]:
import datetime
import hashlib
import random
from Crypto.Cipher import AES

In [ ]:
# Shared (Client and Server only) Secret Information
IV = '*9wqgg$hsakh8!@v' # multiple of 16 bytes
KEY = 'This is a key123This is a key123' # 16, 24 or 32 bytes
BLOCK_SIZE = 64

In [ ]:
from base64 import b64encode, b64decode

def encrypt_block(text):
    global KEY
    cipher = AES.new(KEY, AES.MODE_ECB)
    if len(text) %16!=0:
        while not len(text)%16==0:
            text=text+" "
    encrypted_text =cipher.encrypt(text)
    return b64encode(encrypted_text).decode('UTF-8')

def decrypt_block(text):
    global KEY
    text = b64decode(text) # decode before decryption
    decipher = AES.new(KEY, AES.MODE_ECB)
    return decipher.decrypt(text).decode('UTF-8')

In [ ]:
class Node:
   def __init__(self, data):
      self.left = None
      self.right = None
      self.data = data
      self.encrypted_block = None # Only for leaf nodes
      self.parent = None
      self.timestamp = None
      self.modified = 0
      self.range = [-1, -1] ## Node's range of indices

In [ ]:
def getCurrentTimestamp():
  return datetime.datetime.now()

In [ ]:
def padding_64(text_file, chunk_size=64):
  chunks = [text_file[i:i+chunk_size] for i in range(0, len(text_file), chunk_size)]

  if len(chunks[-1]) < chunk_size:
    last_chunk = chunks[-1].ljust(chunk_size,'=')
    chunks[-1] = last_chunk

  if len(chunks) % 2 != 0:
    chunks.append("="*BLOCK_SIZE)

  return chunks

In [ ]:
# def calcFileBlockHashes(encrypted_file_blocks):
#   hashed_blocks = []
#   for i in encrypted_file_blocks:
#     hasher = hashlib.sha256()
#     hasher.update(str(i).encode('utf-8'))
#     hashed_blocks.append(hasher.digest())
#   return hashed_blocks

def calcFileBlockHashes(encrypted_file_blocks):
  hashed_blocks = []
  for i in encrypted_file_blocks:
    hashed_blocks.append(calcBlockHash(i.encode('utf-8')))
  return hashed_blocks

In [ ]:
def calcBlockHash(block):
  hasher = hashlib.sha256()
  hasher.update(str(block).encode('utf-8'))
  return hasher.digest()

In [ ]:
def upgradeMerkleTree(prev_node, cur_node, low, high, pos, val):
  print("low = ", low, "high = ", high, pos, val)
  if cur_node is None or prev_node is None:
    print("OH NO")

  # print("current ",cur_node.range, prev_node.range)
  if (pos > high or pos < low or low > high):
    return None

  # if not prev_node:
  #   return prev_node

  if (low == high):
    # if prev_node is None:
    #   return prev_node
    cur_node.encrypted_block = val
    cur_node.data = calcBlockHash(val.encode('utf-8'))
    # cur_node.range = prev_node.range
    cur_node.range[0] = low
    cur_node.range[1] = high
    cur_node.timestamp = getCurrentTimestamp()
    cur_node.modified = cur_node.modified + 1
    cur_node.left = None
    cur_node.right = None
    print("$$$$$$$$ Uodated leaf node index: ", low)
    print(cur_node.encrypted_block)
    print(decrypt_block(cur_node.encrypted_block))

    # cur_node = prev_node
    return

  mid = int((low+high)/2)

  if pos<=mid:
    cur_node.right = prev_node.right
    cur_node.left = Node(0) # left side needs to be updated
    cur_node.left.parent = cur_node
    upgradeMerkleTree(prev_node.left, cur_node.left, low, mid, pos, val)
  else:
    cur_node.left = prev_node.left
    cur_node.right = Node(0) # right side needs to be updated
    cur_node.right.parent = cur_node
    upgradeMerkleTree(prev_node.right, cur_node.right, mid+1, high, pos, val)

  #print("%%%%%%%%")
  #print("low ",low, " high ", high, " mid ", mid)
  #print("left: ", cur_node.left.data)
  #print("right: ", cur_node.right.data)
  assert(cur_node != None)

  cur_node.data = calcBlockHash(cur_node.left.data + cur_node.right.data)
  # if (cur_data.left == None and cur_data.right == None):
  # cur_node.range = prev_node.range
  cur_node.range[0] = low
  cur_node.range[1] = high

  cur_node.timestamp = getCurrentTimestamp()
  cur_node.modified = cur_node.modified + 1 # Check prev vs cur node
  return 

In [ ]:
# For creating the first version
def buildMerkleTree(blocks, cur_node, low, high):
  #Base Case: Leaf Node
  if(low == high):
    cur_node.encrypted_block = blocks[low]
    print("Leaf node ", blocks[low])
    cur_node.data = calcBlockHash(blocks[low].encode('utf-8')) #Storing encrypted hashed blocks at leaf nodes, converted to unicode
    cur_node.range[0] = low
    cur_node.range[1] = high
    cur_node.timestamp = getCurrentTimestamp()
    # print("#debug: during build: ", low, blocks[low],  obj_test.decrypt(blocks[low]))
    cur_node.left = None
    cur_node.right = None
    assert(cur_node.encrypted_block == blocks[low])
    return cur_node

  mid = int((low + high)/2);

  # Create and link empty nodes as cur_node's left and right child
  # Save pointer to cur_node at the left and right child
  cur_node.left = Node(0)
  cur_node.right = Node(0)
  cur_node.left.parent = cur_node
  cur_node.right.parent = cur_node

  cur_node.left = buildMerkleTree(blocks, cur_node.left, low, mid);
  cur_node.right = buildMerkleTree(blocks, cur_node.right, mid+1, high);

  # Calculate cur_node's data from left and right child(already calculated by recursion)
  cur_node.data = calcBlockHash(cur_node.left.data + cur_node.right.data) #Hashed
  cur_node.range[0] = low
  cur_node.range[1] = high
  cur_node.timestamp = getCurrentTimestamp()
  return cur_node

In [ ]:
def printMerkleTree(root, save_info):
  if root.left is None and root.right is None:
    #print(root.range[0], root.range[1])
    #print(root.encrypted_block)
    save_info[root.range[0]] = root.encrypted_block
    print(decrypt_block(root.encrypted_block), '\n', decrypt_block(save_info[root.range[0]]), '\n')
    return 
    
  printMerkleTree(root.left, save_info)
  printMerkleTree(root.right, save_info)

In [ ]:
def sanity_test_per_idx(root, base_data):
  # to see, if the encoded strings are properly set 
  # print(root.range, root.left, root.right)
  if root.left is None and root.right is None:
    assert(root.range[0] == root.range[1])
    assert(base_data[root.range[0]] == root.encrypted_block)
    return 
    
  sanity_test_per_idx(root.left, base_data)
  sanity_test_per_idx(root.right, base_data)

In [ ]:
def get_sibling_nodes(node, low, high, pos, sibling_nodes, hashes=True):
  if (pos > high or pos < low or low > high):
    return None
  if node.left == node.right and node.left == None: # reached child node
    return 
  mid = int((low+high)/2)
  if pos<=mid:
    get_sibling_nodes(node.left, low, mid, pos, sibling_nodes)
    if node.right is not None: 
      if hashes is True:
        sibling_nodes.append(['right',node.right.data])
      else:
        sibling_nodes.append(['right',node.right]) # node version
  else:
    get_sibling_nodes(node.right, mid+1, high, pos, sibling_nodes)
    if node.left is not None: 
      if hashes is True:
        sibling_nodes.append(['left',node.left.data])
      else:
        sibling_nodes.append(['right',node.left]) # node version
  return  


In [ ]:
def get_a_leaf_node(node, low, high, pos):
  print(low, high, pos)
  if(low == high):
    assert(node.range[0] == node.range[1] and node.range[0] == pos)
    return node
  mid = int((low+high)/2)
  if(pos<=mid):
    return get_a_leaf_node(node.left, low, mid, pos)
  else:
    return get_a_leaf_node(node.right, mid+1, high, pos)

In [ ]:
# File encryption at user end
text_file = "HITS occurs specifically on Ubuntu's and not other operating systems. How can I free up the port to restart the server? By default, the runserver command starts the development server on the internal IP at port 8000. If you want to change the servers port, pass it as a command-line argument. For instance, this command starts the server on port 8080. General Usage Select the text to format and then execute the Pretty JSON prettify command. For JSON files, format the entire file automatically. Ubuntu is the modern, open source operating system on Linux for the enterprise server, desktop, cloud, and IoT. "

raw_blocks = padding_64(text_file)
print(raw_blocks[0])
print(raw_blocks)
print(len(raw_blocks))

HITS occurs specifically on Ubuntu's and not other operating sys
["HITS occurs specifically on Ubuntu's and not other operating sys", 'tems. How can I free up the port to restart the server? By defau', 'lt, the runserver command starts the development server on the i', 'nternal IP at port 8000. If you want to change the servers port,', ' pass it as a command-line argument. For instance, this command ', 'starts the server on port 8080. General Usage Select the text to', ' format and then execute the Pretty JSON prettify command. For J', 'SON files, format the entire file automatically. Ubuntu is the m', 'odern, open source operating system on Linux for the enterprise ', 'server, desktop, cloud, and IoT. ===============================']
10


In [ ]:
print("raw blocks ", type(raw_blocks[0]), raw_blocks[0])

encrypted_blocks = []
decrypted_blocks = []

for i in raw_blocks:
  encrypted_blocks.append(encrypt_block(i))
  print(encrypted_blocks[-1])
  print('\n')
print(len(encrypted_blocks))
# print("encrypted_blocks ", len(encrypted_blocks), type(encrypted_blocks[0]), encrypted_blocks)

raw blocks  <class 'str'> HITS occurs specifically on Ubuntu's and not other operating sys
dD9yrjh1o4rH3GObQ0wtJaJejJbVYV6skO75Nh3B0V0LzRjv7b8+PerLm4RL3amYIes2AIPAqgRIuIHRG5lGog==


nKQwqk1oaP4a0NfknUWtBY6UYOc86LERwxpxsXnqDZLVDvnP3U9RTYeiKogewuDSlwu3vo0ESr7Y4t+6jHRXQQ==


d99o1REx91Wn8ExlEXYMqXQme9lMgWM+XeG4AX9hyvnFP8urh++mJ5YfjBM7FeZqh/JbQIUNtu+XyF52JKuTAQ==


aU7pMBfbtT4t52aqoStFhEcp7o+OP+mZC4O+pBYV9MfnLE9f8whO2nGZHlDp0tPmB4qn15NhLkwV2NVy7ZhNFg==


Ndo+FRa2cGF21PkACRMGIUNJzjdS/517XftfKKZUDrrnrKpH/5VBy/17TDt9PClETw84wbbuTPpyGyHjWnP5pQ==


JFEht5t1Yn0qZUsH1hu/zHo5Z7IqizMkp0JXd9WR6jaVWYoOAcBvaxwuuJEgbxSVFUnP4a1oaow7+dm+Dkki+g==


s4P7pTbAV4pTtXfJfFcHohdv2GgS4M2VrER7DtCtP+FNOMr3O0SGv4Fl+jqS9NKZ5ZA2Ykf3OwY8XlblgWNaDA==


Znotsuxf0G6bs7XvyhpVSgFjoRwp/irJ2uiDDSxwssoLwartDGjEsQv/Fz3nx9ZrcZX4RlAU69W+sZegEU2ZLg==


Y8M/TN6mhsVCob0j0P7LE0CueD6ZdRpFXCJIfv3R7gGltuXU/AgzeMnf5R9Lzu83ddYn8G6CoeGzgAvbKDkTSA==


qGndCh+kLGlQr6WwyhqGd9gmLrLfJqxuiGUMogjLAiT3nkDDMNvuDcicjFdwPImvZqJniK1fAqF1MQ2YG+JM8g==



In [ ]:
# basic decryption
# obj_test2 = AES.new(KEY, AES.MODE_CBC, IV) 

decrypted_blocks = []
for i in encrypted_blocks:
  decrypted_blocks.append(decrypt_block(i))
  print(decrypted_blocks[-1],'\n')
print("decrypted_blocks ", len(decrypted_blocks))

HITS occurs specifically on Ubuntu's and not other operating sys 

tems. How can I free up the port to restart the server? By defau 

lt, the runserver command starts the development server on the i 

nternal IP at port 8000. If you want to change the servers port, 

 pass it as a command-line argument. For instance, this command  

starts the server on port 8080. General Usage Select the text to 

 format and then execute the Pretty JSON prettify command. For J 

SON files, format the entire file automatically. Ubuntu is the m 

odern, open source operating system on Linux for the enterprise  

server, desktop, cloud, and IoT. =============================== 

decrypted_blocks  10


In [ ]:
version_roots = []
n = len(encrypted_blocks)

# Creating first merkle tree
v0_root = buildMerkleTree(encrypted_blocks, Node(0), 0, n-1)
version_roots.append(v0_root)

Leaf node  dD9yrjh1o4rH3GObQ0wtJaJejJbVYV6skO75Nh3B0V0LzRjv7b8+PerLm4RL3amYIes2AIPAqgRIuIHRG5lGog==
Leaf node  nKQwqk1oaP4a0NfknUWtBY6UYOc86LERwxpxsXnqDZLVDvnP3U9RTYeiKogewuDSlwu3vo0ESr7Y4t+6jHRXQQ==
Leaf node  d99o1REx91Wn8ExlEXYMqXQme9lMgWM+XeG4AX9hyvnFP8urh++mJ5YfjBM7FeZqh/JbQIUNtu+XyF52JKuTAQ==
Leaf node  aU7pMBfbtT4t52aqoStFhEcp7o+OP+mZC4O+pBYV9MfnLE9f8whO2nGZHlDp0tPmB4qn15NhLkwV2NVy7ZhNFg==
Leaf node  Ndo+FRa2cGF21PkACRMGIUNJzjdS/517XftfKKZUDrrnrKpH/5VBy/17TDt9PClETw84wbbuTPpyGyHjWnP5pQ==
Leaf node  JFEht5t1Yn0qZUsH1hu/zHo5Z7IqizMkp0JXd9WR6jaVWYoOAcBvaxwuuJEgbxSVFUnP4a1oaow7+dm+Dkki+g==
Leaf node  s4P7pTbAV4pTtXfJfFcHohdv2GgS4M2VrER7DtCtP+FNOMr3O0SGv4Fl+jqS9NKZ5ZA2Ykf3OwY8XlblgWNaDA==
Leaf node  Znotsuxf0G6bs7XvyhpVSgFjoRwp/irJ2uiDDSxwssoLwartDGjEsQv/Fz3nx9ZrcZX4RlAU69W+sZegEU2ZLg==
Leaf node  Y8M/TN6mhsVCob0j0P7LE0CueD6ZdRpFXCJIfv3R7gGltuXU/AgzeMnf5R9Lzu83ddYn8G6CoeGzgAvbKDkTSA==
Leaf node  qGndCh+kLGlQr6WwyhqGd9gmLrLfJqxuiGUMogjLAiT3nkDDMNvuDcicjFdwPImvZqJniK1fAqF1MQ2YG+JM8g==


In [ ]:
save_info = {}
printMerkleTree(v0_root, save_info)
sanity_test_per_idx(v0_root, encrypted_blocks)

HITS occurs specifically on Ubuntu's and not other operating sys 
 HITS occurs specifically on Ubuntu's and not other operating sys 

tems. How can I free up the port to restart the server? By defau 
 tems. How can I free up the port to restart the server? By defau 

lt, the runserver command starts the development server on the i 
 lt, the runserver command starts the development server on the i 

nternal IP at port 8000. If you want to change the servers port, 
 nternal IP at port 8000. If you want to change the servers port, 

 pass it as a command-line argument. For instance, this command  
  pass it as a command-line argument. For instance, this command  

starts the server on port 8080. General Usage Select the text to 
 starts the server on port 8080. General Usage Select the text to 

 format and then execute the Pretty JSON prettify command. For J 
  format and then execute the Pretty JSON prettify command. For J 

SON files, format the entire file automatically. Ubuntu is the

In [ ]:
for key in save_info:
  print(key)
  print(save_info[key])
  print(encrypted_blocks[key])
  print(decrypted_blocks[key])
  print(decrypt_block(encrypted_blocks[key]))
  print(decrypt_block(save_info[key]))
  print(encrypted_blocks[key] == save_info[key])

0
dD9yrjh1o4rH3GObQ0wtJaJejJbVYV6skO75Nh3B0V0LzRjv7b8+PerLm4RL3amYIes2AIPAqgRIuIHRG5lGog==
dD9yrjh1o4rH3GObQ0wtJaJejJbVYV6skO75Nh3B0V0LzRjv7b8+PerLm4RL3amYIes2AIPAqgRIuIHRG5lGog==
HITS occurs specifically on Ubuntu's and not other operating sys
HITS occurs specifically on Ubuntu's and not other operating sys
HITS occurs specifically on Ubuntu's and not other operating sys
True
1
nKQwqk1oaP4a0NfknUWtBY6UYOc86LERwxpxsXnqDZLVDvnP3U9RTYeiKogewuDSlwu3vo0ESr7Y4t+6jHRXQQ==
nKQwqk1oaP4a0NfknUWtBY6UYOc86LERwxpxsXnqDZLVDvnP3U9RTYeiKogewuDSlwu3vo0ESr7Y4t+6jHRXQQ==
tems. How can I free up the port to restart the server? By defau
tems. How can I free up the port to restart the server? By defau
tems. How can I free up the port to restart the server? By defau
True
2
d99o1REx91Wn8ExlEXYMqXQme9lMgWM+XeG4AX9hyvnFP8urh++mJ5YfjBM7FeZqh/JbQIUNtu+XyF52JKuTAQ==
d99o1REx91Wn8ExlEXYMqXQme9lMgWM+XeG4AX9hyvnFP8urh++mJ5YfjBM7FeZqh/JbQIUNtu+XyF52JKuTAQ==
lt, the runserver command starts the development server on t

In [ ]:
# Update operation

block_text = "This is text".ljust(BLOCK_SIZE,"#")
block_text2 = "This is a Random text".ljust(BLOCK_SIZE,"*")

print(type(block_text), len(block_text), block_text)

changed_data = encrypt_block(block_text)
print(changed_data)
print(decrypt_block(changed_data))

<class 'str'> 64 This is text####################################################
dM49AaOuDE+sSfPar64aqzpqcwr8rOL48lNqtR1vEMQ6anMK/Kzi+PJTarUdbxDEOmpzCvys4vjyU2q1HW8QxA==
This is text####################################################


In [ ]:
def make_copy(list1):
  # to copy lists 
  list2 = []
  for i in range(0, len(list1)):
    list2.append(list1[i])
  return list2 

In [ ]:
v1_root = Node(0)
new_encrypted_blocks = make_copy(encrypted_blocks)
idx_to_update = 2
new_encrypted_blocks[idx_to_update] = changed_data
upgradeMerkleTree(v0_root, v1_root, 0, n-1, idx_to_update, changed_data) #Modifying blocks[1] and creating a new version
version_roots.append(v1_root)
print(len(version_roots))

low =  0 high =  9 2 dM49AaOuDE+sSfPar64aqzpqcwr8rOL48lNqtR1vEMQ6anMK/Kzi+PJTarUdbxDEOmpzCvys4vjyU2q1HW8QxA==
low =  0 high =  4 2 dM49AaOuDE+sSfPar64aqzpqcwr8rOL48lNqtR1vEMQ6anMK/Kzi+PJTarUdbxDEOmpzCvys4vjyU2q1HW8QxA==
low =  0 high =  2 2 dM49AaOuDE+sSfPar64aqzpqcwr8rOL48lNqtR1vEMQ6anMK/Kzi+PJTarUdbxDEOmpzCvys4vjyU2q1HW8QxA==
low =  2 high =  2 2 dM49AaOuDE+sSfPar64aqzpqcwr8rOL48lNqtR1vEMQ6anMK/Kzi+PJTarUdbxDEOmpzCvys4vjyU2q1HW8QxA==
$$$$$$$$ Uodated leaf node index:  2
dM49AaOuDE+sSfPar64aqzpqcwr8rOL48lNqtR1vEMQ6anMK/Kzi+PJTarUdbxDEOmpzCvys4vjyU2q1HW8QxA==
This is text####################################################
2


In [ ]:
save_info = {}
printMerkleTree(v1_root, save_info)
sanity_test_per_idx(v1_root, new_encrypted_blocks)

HITS occurs specifically on Ubuntu's and not other operating sys 
 HITS occurs specifically on Ubuntu's and not other operating sys 

tems. How can I free up the port to restart the server? By defau 
 tems. How can I free up the port to restart the server? By defau 

This is text#################################################### 
 This is text#################################################### 

nternal IP at port 8000. If you want to change the servers port, 
 nternal IP at port 8000. If you want to change the servers port, 

 pass it as a command-line argument. For instance, this command  
  pass it as a command-line argument. For instance, this command  

starts the server on port 8080. General Usage Select the text to 
 starts the server on port 8080. General Usage Select the text to 

 format and then execute the Pretty JSON prettify command. For J 
  format and then execute the Pretty JSON prettify command. For J 

SON files, format the entire file automatically. Ubuntu is the

In [ ]:
def print_sibling_nodes(sibling_nodes):
  for i in range(0, len(sibling_nodes)):
    print("i = ", i, sibling_nodes[i][1].range[0], sibling_nodes[i][1].range[1])

def print_sibling_nodes_hashes(sibling_nodes):
  for i in range(0, len(sibling_nodes)):
    print("i = ", sibling_nodes[i])

In [ ]:
sibling_nodes = []
get_sibling_nodes(node=v0_root, low=0, high=n-1, pos=7 , sibling_nodes=sibling_nodes)

print_sibling_nodes_hashes(sibling_nodes)
#print_sibling_nodes(sibling_nodes)

i =  ['left', b'=\x92\x1c\xf8\x84\xd8\x8e\xa6@\x15\xd1\xb9\x93\xced\x9d\xf2\xfe\xef}\xf2\x02(\r\x12\xce\x1e\x02\xd7\xa7<S']
i =  ['right', b'f\x81oN\xc5\xe8\x92Z-`\xc4\x1f\x90Z\xe0 d]\xbbd\xf0\xa1`)=\xeaO\xb6\xa9%1\n']
i =  ['left', b'\xf7\x1b\x05Z\xec\x91 -!A\xc2\x97\x06\xd1\xd2\x84?\xc5\x91\xa2\x83b\x89\xfb\xbce\x19P>u\xb4\xc4']


In [ ]:
# sibling_nodes_hashes = []
# get_sibling_nodes_hashes(node=v0_root, low=0, high=n-1, pos=1, sibling_nodes_hashes=sibling_nodes_hashes)

# print(sibling_nodes_hashes)

In [ ]:
def calculate_root_from_sib_path(input_hash, sibling_nodes):
  current_hash = input_hash 
  for i in range(0, len(sibling_nodes)):
    if(sibling_nodes[i][0] == 'left'):
      current_hash = calcBlockHash(sibling_nodes[i][1] + current_hash) #Hashed
    else:
      current_hash = calcBlockHash(current_hash+sibling_nodes[i][1]) #Hashed
  return current_hash # will return the updated hash 

In [ ]:
input_hash = calcBlockHash(encrypted_blocks[1].encode('utf-8'))
print(input_hash)
final_hash = calculate_root_from_sib_path(input_hash, sibling_nodes)
print(final_hash)
print(version_roots[0].data)
print(v0_root.data)

b'\x13\xcd\xd0\x1e\x92h\xc5\xb7\x12\x8a\xa4\xb3\xb1\xb9\xafq\xba\xbak\x99\xd6h(\xa8w\xf7\x89\xd2\tsG\xbd'
b'7b)\xc9\x0e3\xb6\xad.\x0e5\x18\xb7\xa6\xae\xb4\xa8\xfcw\xf1<\xb5t\xf1\xda\xef\x128\xdb`\xa6m'
b'\x02\xfb+\xacH\x88\xeb\xd2Q\xbc$&\xec\xf7\x18\xc8!\xe96\x17\xca\xb9`@\xe5\x07&l\x0by\x8a\xe0'
b'\x02\xfb+\xacH\x88\xeb\xd2Q\xbc$&\xec\xf7\x18\xc8!\xe96\x17\xca\xb9`@\xe5\x07&l\x0by\x8a\xe0'


In [ ]:
class ServerSegmentTreeControl:
  # a class to control the persistent segment tree 
  def __init__(self):
    self.version_roots = [] # 0 based indexing
    self.max_number_of_leaves = 0

  def initial_file_upload(self, encrypted_blocks):
    return  self.build_initial_segment_tree(encrypted_blocks=encrypted_blocks)

  def update_existing_file(self, encrypted_block="", block_to_update=0):
    return self.update_segment_tree(encrypted_block, block_to_update)

  def build_initial_segment_tree(self, encrypted_blocks):
    # File encryption at user end
    # getting raw blocks
    #raw_blocks = padding_64(text_file)
    # getting encryption 
    # encrypted_blocks = self.do_encryption(raw_blocks=raw_blocks)
    self.max_number_of_leaves=len(encrypted_blocks)
    print("number of segment tree leaf nodes {}".format(self.max_number_of_leaves))
    # Creating first merkle tree
    v0_root = buildMerkleTree(encrypted_blocks, Node(0), 0, self.max_number_of_leaves-1)
    self.version_roots.append(v0_root)
    return self.version_roots[0]

  def do_encryption(self, raw_blocks):
    encrypted_blocks = []
    for i in raw_blocks:
      encrypted_blocks.append(encrypt_block(i))
    return encrypted_blocks

  def update_segment_tree(self,encrypted_block="", block_to_update=0):
    # getting raw blocks
    #raw_blocks = padding_64(block_text)
    #print(raw_blocks)
    # getting encryption 
    #encrypted_blocks = self.do_encryption(raw_blocks=raw_blocks)
    #print(len(encrypted_blocks))
    # new version root
    self.version_roots.append(Node(0))
    upgradeMerkleTree(prev_node=self.version_roots[-2], cur_node=self.version_roots[-1], low=0, high=self.max_number_of_leaves-1, 
                            pos=block_to_update, 
                            val=encrypted_blocks)
    return self.version_roots[-1]

  def print_tree(self, version_no):
    save_info = {}
    printMerkleTree(root=self.version_roots[version_no], save_info=save_info)

  def challenge_server(self, file_version_no, block_index):
    sibling_nodes = []
    get_sibling_nodes(node=self.version_roots[file_version_no], low=0, high=self.max_number_of_leaves-1, pos=block_index, sibling_nodes=sibling_nodes)
    return sibling_nodes

  def file_retrival(self, file_version_no):
    encrypted_blocks = []
    for i in range(0, self.max_number_of_leaves):
      encrypted_blocks.append(get_a_leaf_node(node=self.version_roots[file_version_no], low=0, high=self.max_number_of_leaves-1, pos=i).data)
    return encrypted_blocks, self.version_roots[file_version_no].data

In [ ]:
server_segment_tree_control = ServerSegmentTreeControl()
# creating initial segemnt tree 
server_segment_tree_control.build_initial_segment_tree("HITS occurs specifically on Ubuntu's and not other operating systems. How can I free up the port to restart the server? By default, the runserver command starts the development server on the internal IP at port 8000. If you want to change the servers port, pass it as a command-line argument. For instance, this command starts the server on port 8080. General Usage Select the text to format and then execute the Pretty JSON prettify command. For JSON files, format the entire file automatically. Ubuntu is the modern, open source operating system on Linux for the enterprise server, desktop, cloud, and IoT.")

In [ ]:
server_segment_tree_control.update_segment_tree(block_text="This is me rizvee", starting_block_to_update=2)

In [ ]:
# server_segment_tree_control.print_tree(version_no=1)
sibling_nodes = server_segment_tree_control.challenge_server(file_version_no=0, block_index=2)
print(sibling_nodes)

[['left', b'L\xff+\x9d\xb5\x81\x84\xc5UG\x94\x87\x0c\x1d\xdc\xfc\xde{y\x91\xf2\xce\xedQ\xb7\xd8\xc0\xb2\xcf76\xc9'], ['right', b'\x87\x16\xc5!\xdd\xefI\xdc\x94\xdb\x81p1v0\x9e\x0f\xce{H\xbfg`~\x0c\x89"\xc8\x93k\xf3<'], ['right', b'\x0b\x96e\xe9\xc1\x89~\x00\xbd\x1b\xbb9\xf9P\xba\xbc\xf6H\xf6N>tH\x8a\xe7\xc4 {\xf6\xd1\x0e\x1b'], ['right', b'\x97\x133\xef\x85e\xe0\x13Q\x0f\tX\x11e# \t\x92\xea#\xfex*\x05\x14\xd8\xc6l;r&l'], ['right', b'\x02X\xb8p\x85\xb6\x9e{\xae*\x8f~\xb9\x87\x8aS\xdfcFrP6\x1eB\xc3\x98)\xb5o,\xc4\x96'], ['right', b'\xde\x1dB\xa8o{\xa3\x19\x08\x90\x1f9i\x12\xc4\xe5\xd0\xce\xb8E\xe0\xf2,%f\xaa\x90\ta\xaa\xbe\xd1'], ['right', b"1\x88\x92\xef\xc1i\xb5\x82\xf2\xe6\xa2\x1b\x8d\xde\x91\xc6v\xf5\x98'A\xbd\xa2b\xe9\tS\x88M\x88Y\xaa"], ['right', b'LJy\xf4!\xd3\xb4\xb5 \x80$\xce\x06w\x15\xbb\xe5\xeeyP\x83\n6}\x88\x1b\x10\x82\xd5*\x10,'], ['right', b'|\xe44\xdf,\xbf$\x91XMX\x0f\x82\xd1#\xa8T\xccCr\xb0\x8c\xa8\tY\xec\xf5=\xb5\xde\xdfg']]


In [ ]:
encrypted_blocks = server_segment_tree_control.file_retrival(file_version_no=0)
print(encrypted_blocks)

Streaming output truncated to the last 5000 lines.
76 151 123
114 151 123
114 132 123
114 123 123
119 123 123
122 123 123
123 123 123
0 607 124
0 303 124
0 151 124
76 151 124
114 151 124
114 132 124
124 132 124
124 128 124
124 126 124
124 125 124
124 124 124
0 607 125
0 303 125
0 151 125
76 151 125
114 151 125
114 132 125
124 132 125
124 128 125
124 126 125
124 125 125
125 125 125
0 607 126
0 303 126
0 151 126
76 151 126
114 151 126
114 132 126
124 132 126
124 128 126
124 126 126
126 126 126
0 607 127
0 303 127
0 151 127
76 151 127
114 151 127
114 132 127
124 132 127
124 128 127
127 128 127
127 127 127
0 607 128
0 303 128
0 151 128
76 151 128
114 151 128
114 132 128
124 132 128
124 128 128
127 128 128
128 128 128
0 607 129
0 303 129
0 151 129
76 151 129
114 151 129
114 132 129
124 132 129
129 132 129
129 130 129
129 129 129
0 607 130
0 303 130
0 151 130
76 151 130
114 151 130
114 132 130
124 132 130
129 132 130
129 130 130
130 130 130
0 607 131
0 303 131
0 151 131
76 151 131
114 151 13